In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.4/112.6 GB disk)


# === MRI DETECTION STAGE ===

In [ ]:
# Load detection model
detect_model = load_model("detect.weights.keras")

# Classes for detection
detect_classes = ['invalid', 'valid']
image_size = (224, 224)

# Path to input image
img_path = "/content/download.jpeg"  # Replace with your image

# Function to predict valid/invalid
def is_valid_image(img_path):
    if not os.path.exists(img_path):
        print("Image not found.")
        return False

    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = detect_model.predict(img_array)
    if prediction.shape[-1] == 1:
        confidence = float(prediction[0][0])
        predicted_class = detect_classes[1] if confidence > 0.5 else detect_classes[0]
        confidence = confidence if predicted_class == 'valid' else 1 - confidence
    else:
        predicted_class = detect_classes[np.argmax(prediction)]
        confidence = np.max(prediction)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Detection: {predicted_class} ({confidence:.2f})")
    plt.show()

    print(f"Detection Result: {predicted_class} (Confidence: {confidence:.2f})")
    return predicted_class == 'valid'

# Run detection
proceed_to_classification = is_valid_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
Detection Result: invalid (Confidence: 1.00)


In [ ]:
# === CLASSIFICATION STAGE ===
if proceed_to_classification:
    from tensorflow.keras.preprocessing import image
    import matplotlib.pyplot as plt
    import numpy as np
    from tensorflow.keras.models import load_model
    import os

    # Load classification model
    classification_model = load_model("classification.weights.keras")
    class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
    image_size = (224, 224)

    def classify_tumor(img_path):
        if not os.path.exists(img_path):
            print("Image not found.")
            return False

        # Load image as PIL (for display) and preprocess copy for model
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        # Predict
        prediction = classification_model.predict(img_array)
        predicted_class = class_names[np.argmax(prediction)]
        confidence = np.max(prediction)

        # ✅ Show the image (this works correctly in Colab)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Classified: {predicted_class} ({confidence:.2f})")
        plt.show()

        print(f"Tumor Type: {predicted_class} (Confidence: {confidence:.2f})")

        return predicted_class != 'notumor'

    proceed_to_segmentation = classify_tumor(img_path)
else:
    proceed_to_segmentation = False
    print("Image is invalid. Skipping classification and segmentation.")


Image is invalid. Skipping classification and segmentation.


In [ ]:
# === SEGMENTATION STAGE ===
if proceed_to_segmentation:
    import torch
    from PIL import Image

    # Load YOLOv5 segmentation model
    seg_model = torch.hub.load('ultralytics/yolov5', 'custom', path='segmentation.pt', force_reload=True)

    # Run inference
    results = seg_model(img_path)

    # Print and show results
    results.print()
    results.show()
else:
    print("No tumor detected. Segmentation skipped.")


No tumor detected. Segmentation skipped.
